# Examples

This Jupyter notebook shows some examples for the different functions of the Julia module `zaf`.

Functions:
- `stft` - [Short-time Fourier transform (STFT)](#stft)
- `istft` - [Inverse STFT](#istft)
- `cqtkernel` - [Constant-Q transform (CQT) kernel](#cqtkernel)
- `cqtspectrogram` - [CQT spectrogram using a CQT kernel](#cqtspectrogram)
- `cqtchromagram` - [CQT chromagram using a CQT kernel](#cqtchromagram)
- `mfcc` - [Mel frequency cepstrum coefficients (MFCCs)](#mfcc)
- `dct` - [Discrete cosine transform (DCT) using the fast Fourier transform (FFT)](#dct)
- `dst` - [Discrete sine transform (DST) using the FFT](#dst)
- `mdct` - [Modified discrete cosine transform (MDCT) using the FFT](#mdct)
- `imdct` - [Inverse MDCT using the FFT](#imdct)

Other:
- `wavread` - Read a WAVE file (using Scipy)
- `wavwrite` - Write a WAVE file (using SciPy)
- `sigplot` - Plot an audio signal in seconds
- `specshow` - Display an audio spectrogram in dB, seconds, and Hz
- `cqtspecshow` - Display a CQT audio spectrogram in dB, seconds, and Hz
- `cqtchromshow` - Display a CQT audio chromagram in seconds and chroma


Author:
- Zafar Rafii
- zafarrafii@gmail.com
- http://zafarrafii.com
- https://github.com/zafarrafii
- https://www.linkedin.com/in/zafarrafii/
- 11/04/20

## <a id="stft"></a>Short-time Fourier transform (STFT)

```
audio_stft = zaf.stft(audio_signal, window_function, step_length)

Inputs:
    audio_signal: audio signal [number_samples,]
    window_function: window function [window_length,]
    step_length: step length in samples
Output:
    audio_stft: audio STFT [window_length, number_frames]
```